In [5]:
import numpy as np
from keras.datasets import mnist, fashion_mnist
import pandas as pd
from keras.layers import Input, Dense, concatenate
from keras.models import Model, load_model
from keras import optimizers
from keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from tqdm import tqdm
from sklearn.decomposition import PCA
import glob
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras import objectives, layers

In [35]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1)) 

In [117]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)

x = layers.MaxPooling2D((8, 8), padding='same')(x)
encoded = layers.Flatten(name="bottleneck")(x)
x = layers.Reshape((1,1,10))(encoded)
# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((8, 8))(x)
x = layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [118]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_276 (Conv2D)          (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_94 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_277 (Conv2D)          (None, 14, 14, 10)        1450      
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 7, 7, 10)          0         
_________________________________________________________________
conv2d_278 (Conv2D)          (None, 7, 7, 10)          910       
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 1, 1, 10)          0         
__________

In [119]:
autoencoder.fit(x_train, x_train,epochs=2)

Epoch 1/5
60000/60000 [==============================] - 15s 246us/step - loss: 0.4468
Epoch 2/5
60000/60000 [==============================] - 14s 230us/step - loss: 0.4153
Epoch 3/5
60000/60000 [==============================] - 14s 228us/step - loss: 0.4112
Epoch 4/5
60000/60000 [==============================] - 14s 230us/step - loss: 0.4092
Epoch 5/5
60000/60000 [==============================] - 14s 231us/step - loss: 0.4078


In [120]:
bottleneck_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("bottleneck").output)

In [121]:
bt_train = bottleneck_model.predict(x_train)

In [122]:
pd.DataFrame(bt_train).to_csv("result_fashion/conv_ae_10_flatten.csv", index=False)